[![Dataflowr](https://raw.githubusercontent.com/dataflowr/website/master/_assets/dataflowr_logo.png)](https://dataflowr.github.io/website/)

# Autodiff and Backpropagation

## Jacobian

Let ${\bf f}:\mathbb{R}^n\to \mathbb{R}^m$, we define its Jacobian as:
\begin{align*}
\newcommand{\bbx}{{\bf x}}
\newcommand{\bbv}{{\bf v}}
\newcommand{\bbw}{{\bf w}}
\newcommand{\bbu}{{\bf u}}
\newcommand{\bbf}{{\bf f}}
\newcommand{\bbg}{{\bf g}}
\frac{\partial \bbf}{\partial \bbx} = J_{\bbf}(\bbx) &= \left( \begin{array}{ccc}
\frac{\partial f_1}{\partial x_1}&\dots& \frac{\partial f_1}{\partial x_n}\\
\vdots&&\vdots\\
\frac{\partial f_m}{\partial x_1}&\dots& \frac{\partial f_m}{\partial x_n}
\end{array}\right)\\
&=\left( \frac{\partial \bbf}{\partial x_1},\dots, \frac{\partial \bbf}{\partial x_n}\right)\\
&=\left(
\begin{array}{c}
\nabla f_1(\bbx)^T\\
\vdots\\
\nabla f_m(x)^T
\end{array}\right)
\end{align*}

Hence the Jacobian $J_{\bbf}(\bbx)\in \mathbb{R}^{m\times n}$ is a linear map from $\mathbb{R}^n$ to $\mathbb{R}^m$ such that for $\bbx,\bbv \in \mathbb{R}^n$ and $h\in \mathbb{R}$:
\begin{align*}
\bbf(\bbx+h\bbv) = \bbf(\bbx) + hJ_{\bbf}(\bbx)\bbv +o(h).
\end{align*}
The term $J_{\bbf}(\bbx)\bbv\in \mathbb{R}^m$ is a Jacobian Vector Product (**JVP**), correponding to the interpretation where the Jacobian is the linear map: $J_{\bbf}(\bbx):\mathbb{R}^n \to \mathbb{R}^m$, where $J_{\bbf}(\bbx)(\bbv)=J_{\bbf}(\bbx)\bbv$.

## Chain composition

In machine learning, we are computing gradient of the loss function with respect to the parameters. In particular, if the parameters are high-dimensional, the loss is a real number. Hence, consider a real-valued function $\bbf:\mathbb{R}^n\stackrel{\bbg_1}{\to}\mathbb{R}^m \stackrel{\bbg_2}{\to}\mathbb{R}^d\stackrel{h}{\to}\mathbb{R}$, so that $\bbf(\bbx) = h(\bbg_2(\bbg_1(\bbx)))\in \mathbb{R}$. We have
\begin{align*}
\underbrace{\nabla\bbf(\bbx)}_{n\times 1}=\underbrace{J_{\bbg_1}(\bbx)^T}_{n\times m}\underbrace{J_{\bbg_2}(\bbg_1(\bbx))^T}_{m\times d}\underbrace{\nabla h(\bbg_2(\bbg_1(\bbx)))}_{d\times 1}.
\end{align*}
To do this computation, if we start from the right so that we start with a matrix times a vector to obtain a vector (of size $m$) and we need to make another matrix times a vector, resulting in $O(nm+md)$ operations. If we start from the left with the matrix-matrix multiplication, we get $O(nmd+nd)$ operations. Hence we see that as soon as $m\approx d$, starting for the right is much more efficient. Note however that doing the computation from the right to the left requires to keep in memory the values of $\bbg_1(\bbx)\in\mathbb{R}^m$, and $\bbx\in \mathbb{R}^n$.

**Backpropagation** is an efficient algorithm computing the gradient "from the right to the left", i.e. backward. In particular, we will need to compute quantities of the form: $J_{\bbf}(\bbx)^T\bbu \in \mathbb{R}^n$ with $\bbu \in\mathbb{R}^m$ which can be rewritten $\bbu^T J_{\bbf}(\bbx)$ which is a Vector Jacobian Product (**VJP**), correponding to the interpretation where the Jacobian is the linear map: $J_{\bbf}(\bbx):\mathbb{R}^n \to \mathbb{R}^m$, composed with the linear map $\bbu:\mathbb{R}^m\to \mathbb{R}$ so that $\bbu^TJ_{\bbf}(\bbx) = \bbu \circ J_{\bbf}(\bbx)$.

**example:** let $\bbf(\bbx, W) = \bbx W\in \mathbb{R}^b$ where $W\in \mathbb{R}^{a\times b}$ and $\bbx\in \mathbb{R}^a$. We clearly have
$$
J_{\bbf}(\bbx) = W^T.
$$
Note that here, we are slightly abusing notations and considering the partial function $\bbx\mapsto \bbf(\bbx, W)$. To see this, we can write $f_j = \sum_{i}x_iW_{ij}$ so that 
$$
\frac{\partial \bbf}{\partial x_i}= \left( W_{i1}\dots W_{ib}\right)^T
$$
Then recall from definitions that
$$
J_{\bbf}(\bbx) = \left( \frac{\partial \bbf}{\partial x_1},\dots, \frac{\partial \bbf}{\partial x_n}\right)=W^T.
$$
Now we clearly have
$$
J_{\bbf}(W) = \bbx \text{ since, } \bbf(\bbx,W+\Delta W) = \bbf(\bbx,W) + \bbx \Delta W.
$$
Note that multiplying $\bbx$ on the right is actually convenient when using broadcasting, i.e. we can take a batch of input vectors of shape $\text{bs}\times a$ without modifying the math above. 

## Implementation

In PyTorch, `torch.autograd` provides classes and functions implementing automatic differentiation of arbitrary scalar valued functions. To create a custom [autograd.Function](https://pytorch.org/docs/stable/autograd.html#torch.autograd.Function), subclass this class and implement the `forward()` and `backward()` static methods. Here is an example:
```python=
class Exp(Function):
    @staticmethod
    def forward(ctx, i):
        result = i.exp()
        ctx.save_for_backward(result)
        return result
    @staticmethod
    def backward(ctx, grad_output):
        result, = ctx.saved_tensors
        return grad_output * result
# Use it by calling the apply method:
output = Exp.apply(input)
```
You can have a look at [Module 2b](https://dataflowr.github.io/website/modules/2b-automatic-differentiation) to learn more about this approach as well as [MLP from scratch](https://dataflowr.github.io/website/homework/1-mlp-from-scratch/).

### Backprop the functional way

Here we will implement in `numpy` a different approach mimicking the functional approach of [JAX](https://jax.readthedocs.io/en/latest/index.html) see [The Autodiff Cookbook](https://jax.readthedocs.io/en/latest/notebooks/autodiff_cookbook.html#).

Each function will take 2 arguments: one being the input `x` and the other being the parameters `w`. For each function, we build 2 **vjp** functions taking as argument a gradient $\bbu$ and corresponding to $J_{\bbf}(\bbx)$ and $J_{\bbf}(\bbw)$ so that these functions return $J_{\bbf}(\bbx)^T \bbu$ and $J_{\bbf}(\bbw)^T \bbu$ respectively. To summarize, for $\bbx \in \mathbb{R}^n$, $\bbw \in \mathbb{R}^d$, and, $\bbf(\bbx,\bbw) \in \mathbb{R}^m$,
\begin{align*}
{\bf jvp}_\bbx(\bbu) &= J_{\bbf}(\bbx)^T \bbu, \text{ with } J_{\bbf}(\bbx)\in\mathbb{R}^{m\times n}, \bbu\in \mathbb{R}^m\\
{\bf jvp}_\bbw(\bbu) &= J_{\bbf}(\bbw)^T \bbu, \text{ with } J_{\bbf}(\bbw)\in\mathbb{R}^{m\times d}, \bbu\in \mathbb{R}^m
\end{align*}
Then backpropagation is simply done by first computing the gradient of the loss and then composing the **vjp** functions in the right order.

### Example: adding bias

We start with the simple example of adding a bias.

In [ ]:
import numpy as np

In [ ]:
def add(x, b):
    return x + b

def add_make_vjp(x, b):
    def vjp(u):
        return u, u
    return vjp

add.make_vjp = add_make_vjp

In [ ]:
rng = np.random.RandomState(0)
x = rng.random((30,2)).astype('float32')
b_source  = np.array([1.])

In [ ]:
xb = add(x,b_source)
np.allclose(xb, x+b_source)

In [ ]:
vjp_add = add.make_vjp(x,b_source)

In [ ]:
grad_x, grad_b = vjp_add(rng.random((30,2)))

In [ ]:
grad_x.shape

### Exercise: dot product and squared loss

Implement the corresponding vjp functions. (Note: we are abusing notation for the squared loss as the target `y` is not a parameter and should not be updated! Moreover, the vjp_{y_pred} function for the squared loss does not depend on its input `u`)

In [ ]:
def dot(x, W):
    return np.dot(x, W)

def dot_make_vjp(x, W):
    def vjp(u):
        #your code
    return vjp

dot.make_vjp = dot_make_vjp

def squared_loss(y_pred, y):
    return np.array([np.sum((y - y_pred) ** 2)])

def squared_loss_make_vjp(y_pred, y):
    def vjp(u):
        # your code
    return vjp

squared_loss.make_vjp = squared_loss_make_vjp

# Setup

As in [02b_linear_reg.ipynb](https://github.com/dataflowr/notebooks/blob/master/Module2/02b_linear_reg.ipynb), our model is:
$$
y_t = 2x^1_t-3x^2_t+1, \quad t\in\{1,\dots,30\}
$$

Our task is given the 'observations' $(x_t,y_t)_{t\in\{1,\dots,30\}}$ to recover the weights $w^1=2, w^2=-3$ and the bias $b = 1$.

In order to do so, we will solve the following optimization problem:
$$
\underset{w^1,w^2,b}{\operatorname{argmin}} \sum_{t=1}^{30} \left(w^1x^1_t+w^2x^2_t+b-y_t\right)^2
$$

In [ ]:
rng = np.random.RandomState(0)
# generate random input data
x = rng.random((30,2)).astype('float32')
# generate labels corresponding to input data x
y = np.dot(x, [2., -3.]) + 1.
y = np.expand_dims(y, axis=1).astype('float32')
w_source = np.array([2., -3.])
b_source  = np.array([1.])

In [ ]:
def create_feed_forward(y, seed=0):
    rng = np.random.RandomState(seed)
    funcs = [dot,add,squared_loss]
    params = [rng.randn(2,1),rng.randn(1),y]
    return funcs, params

### Forward pass

The following function should take a batch of inputs, functions and their parameters and return the final value or all values.

In [ ]:
def evaluate_chain(x, funcs, params, return_all=False):
    all_x = [x]
    #
    # your code
    #

In [ ]:
funcs, params = create_feed_forward(y=y, seed=0)
W, b, _ = params

In [ ]:
xs = evaluate_chain(x, funcs, params, return_all=True)

### Backward pass

The following function should do the forward pass and then the backward pass.

In [ ]:
def backward_diff_chain(x, funcs, params):
    """
    Reverse-mode differentiation of a chain of computations.

    Args:
    x: initial input to the chain.
    funcs: a list of functions of the form func(x, param).
    params: a list of parameters, with len(params) = len(funcs).
    Returns:
    value, vjp_x, all vjp_params
    """
    # Evaluate the feedforward model and store intermediate computations,
    # as they will be needed during the backward pass.
    xs = evaluate_chain(x, funcs, params, return_all=True)
    K = len(funcs)  # Number of functions.
    u = None # the gradient of the loss does not require an input
    # List that will contain the Jacobian of each function w.r.t. parameters.
    J = [None] * K

    #
    # your code
    #

In [ ]:
loss, grad_x, grads = backward_diff_chain(x, funcs, params)

### Optimizer

First compute the update for each parameter and then modify the parameters.

In [ ]:
def optim_SGD(grads, learning_rate = 1e-2):
    #your code
    
def update_params(updates, params):
    # your code

### Training loop

In [ ]:
funcs, params = create_feed_forward(y=y, seed=0)
W, b, _ = params
for epoch in range(10):
    loss, grad_x, grads = backward_diff_chain(x, funcs, params)
    print("progress:", "epoch:", epoch, "loss",loss)
    updates = optim_SGD(grads)
    params = update_params(updates, params)
    
# After training
print("estimation of the parameters:")
print(params[:-1])

### Jax implementation

see [linear_regression_jax.ipynb](https://github.com/dataflowr/notebooks/blob/master/Module2/linear_regression_jax.ipynb)

In [ ]:
import jax
import jax.numpy as jnp
import haiku as hk
import optax
from functools import partial

class config:
    size_out = 1
    w_source = jnp.array(W)
    b_source = jnp.array(b)
    
def _linear(x, config):
    return hk.Linear(config.size_out,w_init=hk.initializers.Constant(config.w_source), b_init=hk.initializers.Constant(config.b_source))(x)

def mse_loss(y_pred, y_t):
    return jax.lax.integer_pow(y_pred - y_t,2).sum()

def loss_fn(x_in, y_t, config):
    return mse_loss(_linear(x=x_in, config=config),y_t)

hk_loss_fn = hk.without_apply_rng(hk.transform(partial(loss_fn, config=config)))
params = hk_loss_fn.init(x_in=x,y_t=y,rng=None)
loss_fn = hk_loss_fn.apply

optimizer = optax.sgd(learning_rate=1e-2)

opt_state = optimizer.init(params)
for epoch in range(10):
    loss, grads = jax.value_and_grad(loss_fn)(params,x_in=x,y_t=y)
    print("progress:", "epoch:", epoch, "loss",loss)
    updates, opt_state = optimizer.update(grads, opt_state, params)
    params = optax.apply_updates(params, updates)
    
# After training
print("estimation of the parameters:")
print(params)

### PyTorch implementation

See [02b_linear_reg.ipynb](https://github.com/dataflowr/notebooks/blob/master/Module2/02b_linear_reg.ipynb)

In [ ]:
import torch

dtype = torch.FloatTensor
w_init_t = torch.from_numpy(W).type(dtype)
b_init_t = torch.from_numpy(b).type(dtype)
x_t = torch.from_numpy(x).type(dtype)
y_t = torch.from_numpy(y).type(dtype)

model = torch.nn.Sequential(torch.nn.Linear(2, 1),)

for m in model.children():
    m.weight.data = w_init_t.T.clone()
    m.bias.data = b_init_t.clone()

loss_fn = torch.nn.MSELoss(reduction='sum')

model.train()

optimizer = torch.optim.SGD(model.parameters(), lr=1e-2)

for epoch in range(10):
    y_pred = model(x_t)
    loss = loss_fn(y_pred, y_t)
    print("progress:", "epoch:", epoch, "loss",loss.item())
    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
# After training
print("estimation of the parameters:")
for param in model.parameters():
    print(param)

[![Dataflowr](https://raw.githubusercontent.com/dataflowr/website/master/_assets/dataflowr_logo.png)](https://dataflowr.github.io/website/)